In [ ]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.nn.functional import binary_cross_entropy
import torch

In [ ]:
old_submission = pd.read_csv('./old_submission.csv').to_numpy()[:,1]
new_submission = pd.read_csv('./old_submission.csv').to_numpy()[:,1]


In [ ]:
plt.hist(old_submission, bins=200)
plt.show()
plt.hist(new_submission, bins=200)
plt.show()

In [ ]:
binary_cross_entropy(torch.tensor(old_submission, dtype=torch.float32), torch.tensor(new_submission, dtype=torch.float32))

In [ ]:
res = np.copy(old_submission)

In [ ]:
res[res < 0.05] = 0
res[res > 0.9] = 1

In [ ]:
plt.hist(res, bins=200)
plt.show()

In [ ]:
from typing import Union
import tqdm, os
def generate_submission(output_dir: str, pred: Union[np.ndarray, torch.Tensor], tag: str = ""):
    if not (len(pred.shape) == 1 or (len(pred.shape) == 2 and pred.shape[1] == 1)):
        print(f"Expect pred to be a vector")
    if not pred.shape[0] == 106692:
        print(f"Expect pred to have length 106692 but have {pred.shape[0]}")

    if isinstance(pred, torch.Tensor):
        pred = pred.detach().cpu().numpy()

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    results = zip(range(len(pred)), pred)
    with tqdm.tqdm(range(len(pred))) as pbar:
        with open(os.path.join(output_dir, f'{"_".join([tag, "submission"])}.csv' if len(tag) > 0 else 'submission.csv'), 'w') as f:
            f.write('id,predicted\n')
            for idx, row in enumerate(results):
                f.write(f"{row[0]},{row[1]}\n")
                if idx % 1e3 == 0: pbar.update(1e3)

In [ ]:
generate_submission('./', res, 'clipped')